In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import kstest
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm

In [ ]:
class Forward_NeuralNet(nn.Module):
    '''A neural network that only do feedforward.'''
    def __init__(self, 
                layers: list[int],
                activation: nn.Module=None,
                weight_init: callable=None,
                random_state=None) -> None:

        super().__init__()

        if random_state is not None:
            torch.manual_seed(random_state)
        
        self.layers = layers
        self.activation = activation
        self.weight_init = weight_init

        # Construct the layers
        Layers = []
        inputs = layers[0]
        for outputs in layers[1:-1]:
            Layers.append(nn.Linear(inputs, outputs))
            Layers.append(activation)
            inputs = outputs

        Layers.append(nn.Linear(layers[-2], layers[-1]))
        self.network = nn.Sequential(*Layers)

        # Inititalize weights
        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.network:
            if isinstance(m, nn.Linear):
                self.weight_init(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def forward(self, dataloader: torch.utils.data.DataLoader, device: torch.device) -> torch.Tensor:
        self.to(device)
        self.eval()
        outputs = []
        with torch.inference_mode():
            for X in dataloader:
                X = X[0].to(device)
                y = self.network(X)
                outputs.append(y)
        return torch.cat(outputs, dim=0) if outputs else torch.Tensor()

In [ ]:
def generate_stacked_one_hot(size, index, N):
    # Create the one-hot vector
    one_hot_vector = torch.zeros(size, dtype=torch.float)
    one_hot_vector[index] = 1.0
    # Stack the one-hot vector N times
    stacked_tensor = one_hot_vector.repeat(N, 1)
    return stacked_tensor

generate_stacked_one_hot(3,1,5)

# No hidden layers

## Continuous inputs

In [ ]:
weights = [nn.init.xavier_uniform_, nn.init.xavier_normal_, nn.init.kaiming_uniform_, nn.init.kaiming_normal_, nn.init.orthogonal_]
result = []
N=10000
for n_input in tqdm(range(2, 4)):
    layers = []
    layers.append(n_input)
    for dist in ['uniform', 'normal']:
        if dist == 'uniform':
            X = torch.rand((N, n_input))
        else:
            X = torch.randn((N, n_input))
        dataset = TensorDataset(X)
        dataloader = DataLoader(dataset, batch_size=N)
        for n_output in range(1, 4):
            for weight in weights:
                params = {'n_input': n_input, 'input_distribution':dist, 'n_output': n_output, 'weight': weight.__name__.rstrip('_')}
                all_outputs = []
                for i in range(N):
                    model = Forward_NeuralNet([n_input, n_output], weight_init=weight)
                    output = model.forward(dataloader, 'mps').to('cpu').numpy()
                    all_outputs.append(output)
                all_outputs = np.concatenate(all_outputs, axis=0)
                result.append((output, params))

In [ ]:
fig = px.histogram(np.random.randn(10000))
fig.show()

In [ ]:
for i in result:
    print(i[1])
    for j in range(i[0].shape[1]): 
        fig = px.histogram(i[0].T[j], histnorm='probability density')
        fig.update_layout(showlegend=False, title=f'output_{j}')
        fig.show()
        print(f'mean: {np.mean(i[0].T[j])}, median: {np.median(i[0].T[j])}, variance: {np.var(i[0].T[j])}')
    print('\n')